In [2]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.21.0.4  70.25 KiB  16      66.2%             739395b2-9b2a-4828-a1a2-a85a7e358b24  rack1
UN  172.21.0.2  70.25 KiB  16      62.4%             d9d860d3-d1f1-4282-85f4-efd30b5b1258  rack1
UN  172.21.0.3  70.25 KiB  16      71.4%             b6057742-477a-4263-8869-1dc8926cb85e  rack1



In [13]:
#Connect to the Cassandra cluster
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [14]:
#q1
#drop a weather keyspace if it already exists
cass.execute("drop keyspace if exists weather;")
#create a weather keyspace with 3x replication
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3};")
#inside weather, create a station_record type containing two ints: tmin and tmax
cass.execute("""
CREATE TYPE
weather.station_record (
    tmin int,
    tmax int
)
""")
#inside weather, create a stations table
cass.execute("""
CREATE TABLE weather.stations (
    id text,
    name TEXT STATIC,
    date date,
    record station_record,
    state TEXT,
    PRIMARY KEY (id,date)
) WITH CLUSTERING ORDER BY (date ASC)
""")

print(cass.execute("describe table weather.stations ").one().create_statement)

In [23]:
#Create a local Spark session
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2872f65d-19ba-42e3-88ad-b93051b36812;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [24]:
sc = spark.sparkContext # for interacting directly with RDDs

In [ ]:
#q2

In [26]:
df = spark.read.text("ghcnd-stations.txt")

In [28]:
pandas_df=df.toPandas()

In [34]:
pandas_df["station"] = pandas_df["value"].str[:11]

In [63]:
from pyspark.sql.functions import col, expr,trim

In [64]:
df2 = df.withColumn("station", expr("substring(value, 0, 11)")).withColumn(
     "STATE",expr("substring(value,38,40)")).withColumn(
     "NAME",expr("substring(value, 41,71)"))
df2

DataFrame[value: string, station: string, STATE: string, NAME: string]

In [69]:
df2=df2.filter(col("STATE")=="BEITBRIDGEZ")

In [70]:
df2.toPandas()

,value,station,STATE,NAME
